<a href="https://colab.research.google.com/github/armlabstanford/jparse/blob/main/jparse_collab_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# J-PARSE: Jacobian-based Projection Algorithm for Resolving Singularities Effectively in Inverse Kinematic Control of Serial Manipulators


### Planar Link Manipulator Task Space control
The kinematic velocity relationship between joint $q$ and end-effector $p$ is:
$${\bf \dot{p}} = J(q) {\bf \dot{q}}$$

For a two-link and three-link planar manipulator whose link lengths are:

| Link | Length |
| :-----: | :-----:  |
| $l_1$ | 1m |
| $l_2$ | 1m |
| $l_3$ | 1m |

The origin of the planar manipulator is at the origin. The end-effector must track a point that follows a circle whose center is at $(x,y)=(1,0)$, and whose radius is specified. The target position can move around the circle at a frequency of $\omega$.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display
# Simulation parameters
dt = 0.05            # time step [s]
T_total = 20         # total simulation time [s]
time_steps = int(T_total/dt)
time = np.linspace(0, T_total, time_steps)

# Robot parameters
l1 = 1.0             # length of link 1 [m]
l2 = 1.0             # length of link 2 [m]

# Desired circular trajectory parameters
center = np.array([1.0, 0.0])   # center of the circle
omega = 0.25                     # angular speed for the desired trajectory [rad/s]

# Pre-allocate storage for simulation data
theta1_hist = np.zeros(time_steps)
theta2_hist = np.zeros(time_steps)
x_hist = np.zeros(time_steps)
y_hist = np.zeros(time_steps)
xd_hist = np.zeros(time_steps)
yd_hist = np.zeros(time_steps)

# INITIAL CONDITIONS: starting joint angles (could be chosen arbitrarily)
theta1 = np.pi/4
theta2 = np.pi/4


### Set Radius to visualize JParse performance

In [ ]:
singularity_bool = True
if singularity_bool == True:
  radius = 2.0 #show ability to handle singular poses
else:
  radius = 0.5 #show ability to handle non-singular poses


## J-PARSE Class

In [ ]:

class JParseClass(object):

    def svd_compose(self,U,S,Vt):
        """
        This function takes SVD: U,S,V and recomposes them for J
        """
        Zero_concat = np.zeros((U.shape[0],Vt.shape[0]-len(S)))
        Sfull = np.zeros((U.shape[1],Vt.shape[0]))
        for row in range(Sfull.shape[0]):
            for col in range(Sfull.shape[1]):
              if row == col:
                  if row < len(S):
                      Sfull[row,col] = S[row]
        J_new =np.matrix(U)*Sfull*np.matrix(Vt)
        return J_new

    def JParse(self, J=None, jac_nullspace_bool=False, gamma=0.1, singular_direction_gain_position=1):
        """
        input: Jacobian J (m x n) numpy matrix
        output: J_parse (n x m) numpy matrix
        """

        #Perform the SVD decomposition of the jacobian
        U, S, Vt = np.linalg.svd(J)
        #Find the adjusted condition number
        sigma_max = np.max(S)
        adjusted_condition_numbers = [sig / sigma_max for sig in S]

        #Find the projection Jacobian
        U_new_proj = []
        S_new_proj = []
        for col in range(len(S)):
            if S[col] > gamma*sigma_max:
                #Singular row
                U_new_proj.append(np.matrix(U[:,col]).T)
                S_new_proj.append(S[col])
        U_new_proj = np.concatenate(U_new_proj,axis=1) #Careful which numpy version is being used for this!!!!!
        J_proj = self.svd_compose(U_new_proj, S_new_proj, Vt)

        #Find the safety jacboian
        S_new_safety = [s if (s/sigma_max) > gamma else gamma*sigma_max for s in S]
        J_safety = self.svd_compose(U,S_new_safety,Vt)

        #Find the singular direction projection components
        U_new_sing = []
        Phi = [] #these will be the ratio of s_i/s_max
        set_empty_bool = True
        for col in range(len(S)):
            if adjusted_condition_numbers[col] <= gamma:
                set_empty_bool = False
                U_new_sing.append(np.matrix(U[:,col]).T)
                Phi.append(adjusted_condition_numbers[col]/gamma) #division by gamma for s/(s_max * gamma), gives smooth transition for Kp =1.0;

        #set an empty Phi_singular matrix, populate if there were any adjusted
        #condition numbers below the threshold
        Phi_singular = np.zeros(U.shape) #initialize the singular projection matrix

        if set_empty_bool == False:
            #construct the new U, as there were singular directions
            U_new_sing = np.matrix(np.concatenate(U_new_sing,axis=1)) #Careful which numpy version is being used for this!!!!!
            Phi_mat = np.matrix(np.diag(Phi))
            gains = np.array([singular_direction_gain_position]*2, dtype=float)
            Kp_singular = np.diag(gains)
            # Now put it all together:
            Phi_singular = U_new_sing @ Phi_mat @ U_new_sing.T @ Kp_singular

        #Obtain psuedo-inverse of the safety jacobian and the projection jacobian
        J_safety_pinv = np.linalg.pinv(J_safety)
        J_proj_pinv = np.linalg.pinv(J_proj)

        if set_empty_bool == False:
            J_parse = J_safety_pinv @ J_proj @ J_proj_pinv + J_safety_pinv @ Phi_singular
        else:
            J_parse = J_safety_pinv @ J_proj @ J_proj_pinv

        if jac_nullspace_bool == True:
            #Find the nullspace of the jacobian
            J_safety_nullspace = np.eye(J_safety.shape[1]) - J_safety_pinv @ J_safety
            return J_parse, J_safety_nullspace

        return J_parse

In [ ]:
def forward_kinematics(theta1, theta2):
    # Compute the (x, y) position of the end-effector.
    # Placeholder (using the standard 2-link planar kinematics):
    x = l1 * np.cos(theta1) + l2 * np.cos(theta1 + theta2)
    y = l1 * np.sin(theta1) + l2 * np.sin(theta1 + theta2)
    return x, y

def jacobian(theta1, theta2):
    # Compute the 2x2 Jacobian matrix for the 2-link manipulator.
    # Placeholder implementation:
    J11 = -l1 * np.sin(theta1) - l2 * np.sin(theta1 + theta2)
    J12 = -l2 * np.sin(theta1 + theta2)
    J21 =  l1 * np.cos(theta1) + l2 * np.cos(theta1 + theta2)
    J22 =  l2 * np.cos(theta1 + theta2)
    J = np.array([[J11, J12],
                  [J21, J22]])
    return J

def control_law(x_current, y_current, x_des, y_des, theta1, theta2, jparse_cls_obj=None):
    # Here is an example implementation using a proportional controller.
    k = 1.5  # proportional gain; adjust as needed
    error = np.array([x_des - x_current, y_des - y_current])
    # Desired Cartesian velocity
    v_des = k * error
    # Compute Jacobian and use pseudoinverse to get joint velocities
    J = jacobian(theta1, theta2)
    J_parse = jparse_cls_obj.JParse(J)
    dq = J_parse @ v_des
    return dq


### Simulation and Animation

In [ ]:
# Main simulation loop

J_parse_cls = JParseClass() #instantiate JPARSE class

for i, t in enumerate(time):
    # Desired end-effector position (circle)
    x_des = center[0] + radius * np.cos(omega * t)
    y_des = center[1] + radius * np.sin(omega * t)

    # Record desired trajectory
    xd_hist[i] = x_des
    yd_hist[i] = y_des

    # Get current end-effector position using forward kinematics
    x_current, y_current = forward_kinematics(theta1, theta2)
    x_hist[i] = x_current
    y_hist[i] = y_current

    # Compute joint velocities using control law
    dq = control_law(x_current, y_current, x_des, y_des, theta1, theta2, jparse_cls_obj=J_parse_cls)

    # Update joint angles
    theta1 += dq.item(0)* dt
    theta2 += dq.item(1)* dt

    # Record joint angles
    theta1_hist[i] = theta1
    theta2_hist[i] = theta2

# 1. Animate the robot arm

# Set up the figure and axis for animation
fig_anim, ax_anim = plt.subplots()
ax_anim.set_xlim(-2, 2)
ax_anim.set_ylim(-2, 2)
ax_anim.set_aspect('equal')
ax_anim.set_title("2-Link Robot Arm Tracking a Circle")

# Lines for links and points for joints
link_line, = ax_anim.plot([], [], 'o-', lw=4)
target_point, = ax_anim.plot([], [], 'rx', markersize=10)

def init():
    link_line.set_data([], [])
    target_point.set_data([], [])
    return link_line, target_point

def animate(i):
    # Compute joint positions based on current angles
    theta1_i = theta1_hist[i]
    theta2_i = theta2_hist[i]
    joint1 = np.array([l1 * np.cos(theta1_i), l1 * np.sin(theta1_i)])
    joint2 = joint1 + np.array([l2 * np.cos(theta1_i + theta2_i),
                                 l2 * np.sin(theta1_i + theta2_i)])
    # Link positions: base -> joint1 -> end-effector
    x_data = [0, joint1[0], joint2[0]]
    y_data = [0, joint1[1], joint2[1]]
    link_line.set_data(x_data, y_data)
    # Update the desired target point with sequence values
    target_point.set_data([xd_hist[i]], [yd_hist[i]])
    return link_line, target_point

ani = animation.FuncAnimation(fig_anim, animate, frames=len(time),
                              init_func=init, interval=50, blit=True)

# Display the animation as an HTML5 video (Colab compatible)
display(HTML(ani.to_jshtml()))

### Now make the necessary *plots*

In [ ]:

# 2. Plot x(t) and y(t) vs. time
fig1, ax1 = plt.subplots()
ax1.plot(time, x_hist, label='x(t) actual', linewidth=2)
ax1.plot(time, xd_hist, '--', label='x(t) desired', linewidth=2)
ax1.plot(time, y_hist, label='y(t) actual', linewidth=2)
ax1.plot(time, yd_hist, '--', label='y(t) desired', linewidth=2)
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Position [m]")
ax1.set_title("End-Effector Position vs. Time")
ax1.legend()
ax1.grid(True)
plt.show()

# 3. Plot the (x,y) trajectory in 2D
fig2, ax2 = plt.subplots()
ax2.plot(x_hist, y_hist, label='Actual Trajectory', linewidth=2)
ax2.plot(xd_hist, yd_hist, '--', label='Desired Circle', linewidth=2)
ax2.set_xlabel("x [m]")
ax2.set_ylabel("y [m]")
ax2.set_title("End-Effector Trajectory")
ax2.legend()
ax2.axis('equal')
ax2.grid(True)
plt.show()

# 4. Plot the two joint angles vs. time
fig3, ax3 = plt.subplots()
ax3.plot(time, theta1_hist, label=r'$\theta_1(t)$', linewidth=2)
ax3.plot(time, theta2_hist, label=r'$\theta_2(t)$', linewidth=2)
ax3.set_xlabel("Time [s]")
ax3.set_ylabel("Joint Angles [rad]")
ax3.set_title("Joint Angles vs. Time")
ax3.legend()
ax3.grid(True)
plt.show()

### Download plots and video

In [ ]:
# --- Save the static plots as PNG files ---
fig1.savefig('end_effector_position_vs_time.png')
fig2.savefig('end_effector_trajectory.png')
fig3.savefig('joint_angles_vs_time.png')

# --- Save the animation as an MP4 video ---
ani.save('robot_arm_animation.mp4', writer='ffmpeg')

# --- Download the files ---
from google.colab import files
files.download('end_effector_position_vs_time.png')
files.download('end_effector_trajectory.png')
files.download('joint_angles_vs_time.png')
files.download('robot_arm_animation.mp4')

# 3 Link Manipulator

In [ ]:
# --- Import Libraries ---
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display

# ======================
# Global Simulation Parameters
dt = 0.05            # Time step [s]
T_total = 30         # Total simulation time [s]
time_steps = int(T_total/dt)
time = np.linspace(0, T_total, time_steps)

# Robot Parameters (3-Link)
l1 = 1.0  # Link 1 length [m]
l2 = 1.0  # Link 2 length [m]
l3 = 1.0  # Link 3 length [m]

In [ ]:

def forward_kinematics(q1, q2, q3):
    """
    Compute the end-effector (x,y) position given three joint angles.

    Parameters:
        q1, q2, q3: Joint angles [rad]

    Returns:
        x, y: End-effector coordinates
    """
    x = l1 * np.cos(q1) + l2 * np.cos(q1 + q2) + l3 * np.cos(q1 + q2 + q3)
    y = l1 * np.sin(q1) + l2 * np.sin(q1 + q2) + l3 * np.sin(q1 + q2 + q3)
    return x, y

def jacobian(q1, q2, q3):
    """
    Compute the 2x3 Jacobian matrix for the 3-link planar manipulator.

    Parameters:
        q1, q2, q3: Joint angles [rad]

    Returns:
        J: 2x3 Jacobian matrix
    """
    J11 = -l1*np.sin(q1) - l2*np.sin(q1+q2) - l3*np.sin(q1+q2+q3)
    J12 = -l2*np.sin(q1+q2) - l3*np.sin(q1+q2+q3)
    J13 = -l3*np.sin(q1+q2+q3)
    J21 =  l1*np.cos(q1) + l2*np.cos(q1+q2) + l3*np.cos(q1+q2+q3)
    J22 =  l2*np.cos(q1+q2) + l3*np.cos(q1+q2+q3)
    J23 =  l3*np.cos(q1+q2+q3)
    J = np.array([[J11, J12, J13],
                  [J21, J22, J23]])
    return J

def control_law(x_current, y_current, x_des, y_des, q1, q2, q3, jacobian_type="JPARSE", jparse_cls_obj=None):
    """
    Compute the joint velocities needed to reduce the end-effector position error.
    Uses a simple proportional controller and the pseudoinverse of the Jacobian.

    Parameters:
        x_current, y_current: Current end-effector position
        x_des, y_des: Desired end-effector position
        q1, q2, q3: Current joint angles
        jacobian_type: Type of Jacobian to use for control ("Jpinv" or "J_dls")

    Returns:
        dq1, dq2, dq3: Joint velocities
    """

    k = 1.5  # Proportional gain
    error = np.array([x_des - x_current, y_des - y_current])
    v_des = k * error  # Desired Cartesian velocity
    J = jacobian(q1, q2, q3)
    if jacobian_type == "Jpinv":
      #1. Use traditinoal Jacobian pseudo-inverse
      dq = np.linalg.pinv(J) @ v_des
    elif jacobian_type == "J_dls":
      #2. Use Damped Least Squares
      lambda_dls = 0.01
      J_dls = J.T @ np.linalg.inv((J @ J.T + lambda_dls * np.eye(2)))
      dq = J_dls @ v_des
    elif jacobian_type == "JPARSE":
      J_parse = jparse_cls_obj.JParse(J)
      dq = J_parse @ v_des
      ddq = [dq.item(0),dq.item(1),dq.item(2)]
      dq = ddq
    return dq[0], dq[1], dq[2]




Simulation Kinematics

In [ ]:
# ======================
# Problem 1: Trajectory Tracking
# ======================
# In Problem 1, the end-effector must track a time-varying circular trajectory.
# Two circles are considered:
#  • Part A: The desired circle is fully reachable.
#  • Part B: The desired circle has a large radius so that parts of the trajectory are challenging to track.
#
# ----------------------
# Define Trajectory Parameters
# ----------------------
# Part A: Fully reachable circle
circleA_center = np.array([1.5, 0.0])
circleA_radius = 0.5
omegaA = 0.5  # Angular speed [rad/s]

# Part B: Partially reachable circle
circleB_center = np.array([1.0, 0.0])
circleB_radius = 2.5  # Larger radius challenges the workspace limits
omegaB = 0.5

# ----------------------
# Simulation for Problem 1, Part A
# ----------------------
# Preallocate storage for simulation data for Part A
q1_hist_A = np.zeros(time_steps)
q2_hist_A = np.zeros(time_steps)
q3_hist_A = np.zeros(time_steps)
x_hist_A = np.zeros(time_steps)
y_hist_A = np.zeros(time_steps)
xd_hist_A = np.zeros(time_steps)
yd_hist_A = np.zeros(time_steps)

# Initial joint angles (can be chosen arbitrarily)
q1_A = np.pi/4
q2_A = np.pi/4
q3_A = np.pi/4

J_parse_cls = JParseClass() #instantiate JPARSE class


for i, t in enumerate(time):
    # Compute desired end-effector position for the circle trajectory:
    x_des = circleA_center[0] + circleA_radius * np.cos(omegaA * t)
    y_des = circleA_center[1] + circleA_radius * np.sin(omegaA * t)
    xd_hist_A[i] = x_des
    yd_hist_A[i] = y_des

    # Get current end-effector position using forward kinematics
    x_current, y_current = forward_kinematics(q1_A, q2_A, q3_A)
    x_hist_A[i] = x_current
    y_hist_A[i] = y_current

    # Compute joint velocities using the control law
    dq1, dq2, dq3 = control_law(x_current, y_current, x_des, y_des, q1_A, q2_A, q3_A, jparse_cls_obj=J_parse_cls)

    # Update joint angles based on the velocity-controlled system
    q1_A += dq1 * dt
    q2_A += dq2 * dt
    q3_A += dq3 * dt

    q1_hist_A[i] = q1_A
    q2_hist_A[i] = q2_A
    q3_hist_A[i] = q3_A

# ----------------------
# Simulation for Problem 1, Part B
# ----------------------
# Preallocate storage for simulation data for Part B
q1_hist_B = np.zeros(time_steps)
q2_hist_B = np.zeros(time_steps)
q3_hist_B = np.zeros(time_steps)
x_hist_B = np.zeros(time_steps)
y_hist_B = np.zeros(time_steps)
xd_hist_B = np.zeros(time_steps)
yd_hist_B = np.zeros(time_steps)

# Initial joint angles for Part B
q1_B = np.pi/4
q2_B = np.pi/4
q3_B = np.pi/4

for i, t in enumerate(time):
    # Compute desired end-effector position for the larger circle:
    x_des = circleB_center[0] + circleB_radius * np.cos(omegaB * t)
    y_des = circleB_center[1] + circleB_radius * np.sin(omegaB * t)
    xd_hist_B[i] = x_des
    yd_hist_B[i] = y_des

    # Get current end-effector position
    x_current, y_current = forward_kinematics(q1_B, q2_B, q3_B)
    x_hist_B[i] = x_current
    y_hist_B[i] = y_current

    # Compute joint velocities using the same control law
    dq1, dq2, dq3 = control_law(x_current, y_current, x_des, y_des, q1_B, q2_B, q3_B, jparse_cls_obj=J_parse_cls)

    # Update joint angles
    q1_B += dq1 * dt
    q2_B += dq2 * dt
    q3_B += dq3 * dt

    q1_hist_B[i] = q1_B
    q2_hist_B[i] = q2_B
    q3_hist_B[i] = q3_B

## 3-link animation path A

In [ ]:

# ----------------------
# Animate the 3-Link Manipulator (Problem 1, Part A)
# ----------------------
fig_anim_A, ax_anim_A = plt.subplots()
ax_anim_A.set_xlim(-3, 3)
ax_anim_A.set_ylim(-3, 3)
ax_anim_A.set_aspect('equal')
ax_anim_A.set_title("3-Link Manipulator Tracking a Reachable Circle (Part A)")

# Plot elements: manipulator links and target point
link_line_A, = ax_anim_A.plot([], [], 'o-', lw=4)
target_point_A, = ax_anim_A.plot([], [], 'rx', markersize=10)

def init_A():
    link_line_A.set_data([], [])
    target_point_A.set_data([], [])
    return link_line_A, target_point_A

def animate_A(i):
    # Compute the positions of each joint:
    joint1 = np.array([l1 * np.cos(q1_hist_A[i]), l1 * np.sin(q1_hist_A[i])])
    joint2 = joint1 + np.array([l2 * np.cos(q1_hist_A[i] + q2_hist_A[i]),
                                 l2 * np.sin(q1_hist_A[i] + q2_hist_A[i])])
    joint3 = joint2 + np.array([l3 * np.cos(q1_hist_A[i] + q2_hist_A[i] + q3_hist_A[i]),
                                 l3 * np.sin(q1_hist_A[i] + q2_hist_A[i] + q3_hist_A[i])])
    # Create a list of positions from the base (0,0) to the end-effector
    x_data = [0, joint1[0], joint2[0], joint3[0]]
    y_data = [0, joint1[1], joint2[1], joint3[1]]
    link_line_A.set_data(x_data, y_data)
    target_point_A.set_data([xd_hist_A[i]], [yd_hist_A[i]])
    return link_line_A, target_point_A

ani_A = animation.FuncAnimation(fig_anim_A, animate_A, frames=len(time),
                                init_func=init_A, interval=50, blit=True)
display(HTML(ani_A.to_jshtml()))

# Plot end-effector positions vs. time (Problem 1, Part A)
fig1_A, ax1_A = plt.subplots()
ax1_A.plot(time, x_hist_A, label='x(t) actual', linewidth=2)
ax1_A.plot(time, xd_hist_A, '--', label='x(t) desired', linewidth=2)
ax1_A.plot(time, y_hist_A, label='y(t) actual', linewidth=2)
ax1_A.plot(time, yd_hist_A, '--', label='y(t) desired', linewidth=2)
ax1_A.set_xlabel("Time [s]")
ax1_A.set_ylabel("Position [m]")
ax1_A.set_title("End-Effector Position vs. Time (Part A), using Jacobian ?? (DLS or Jpinv)")
ax1_A.legend()
ax1_A.grid(True)
plt.show()

# Plot the 2D trajectory for Part A
fig2_A, ax2_A = plt.subplots()
ax2_A.plot(x_hist_A, y_hist_A, label='Actual Trajectory', linewidth=2)
ax2_A.plot(xd_hist_A, yd_hist_A, '--', label='Desired Trajectory', linewidth=2)
ax2_A.set_xlabel("x [m]")
ax2_A.set_ylabel("y [m]")
ax2_A.set_title("End-Effector Trajectory (Part A)")
ax2_A.legend()
ax2_A.axis('equal')
ax2_A.grid(True)
plt.show()

## Animate tracking path B


In [ ]:
# ----------------------
# Animate the 3-Link Manipulator (Problem 1, Part B)
# ----------------------
fig_anim_B, ax_anim_B = plt.subplots()
ax_anim_B.set_xlim(-3, 3)
ax_anim_B.set_ylim(-3, 3)
ax_anim_B.set_aspect('equal')
ax_anim_B.set_title("3-Link Manipulator Tracking a Partially Reachable Circle (Part B)")

# Plot elements: manipulator links and target (desired) point
link_line_B, = ax_anim_B.plot([], [], 'o-', lw=4)
target_point_B, = ax_anim_B.plot([], [], 'rx', markersize=10)

def init_B():
    link_line_B.set_data([], [])
    target_point_B.set_data([], [])
    return link_line_B, target_point_B

def animate_B(i):
    # Compute positions of each joint using the Part B joint histories
    joint1 = np.array([l1 * np.cos(q1_hist_B[i]), l1 * np.sin(q1_hist_B[i])])
    joint2 = joint1 + np.array([l2 * np.cos(q1_hist_B[i] + q2_hist_B[i]),
                                 l2 * np.sin(q1_hist_B[i] + q2_hist_B[i])])
    joint3 = joint2 + np.array([l3 * np.cos(q1_hist_B[i] + q2_hist_B[i] + q3_hist_B[i]),
                                 l3 * np.sin(q1_hist_B[i] + q2_hist_B[i] + q3_hist_B[i])])
    # Create list of positions from the base (0,0) to the end-effector
    x_data = [0, joint1[0], joint2[0], joint3[0]]
    y_data = [0, joint1[1], joint2[1], joint3[1]]
    link_line_B.set_data(x_data, y_data)
    # Update the desired target point for Part B trajectory
    target_point_B.set_data([xd_hist_B[i]], [yd_hist_B[i]])
    return link_line_B, target_point_B

ani_B = animation.FuncAnimation(fig_anim_B, animate_B, frames=len(time),
                                init_func=init_B, interval=50, blit=True)
display(HTML(ani_B.to_jshtml()))

# ----------------------
# Plot end-effector positions vs. time (Part B)
# ----------------------
fig1_B, ax1_B = plt.subplots()
ax1_B.plot(time, x_hist_B, label='x(t) actual', linewidth=2)
ax1_B.plot(time, xd_hist_B, '--', label='x(t) desired', linewidth=2)
ax1_B.plot(time, y_hist_B, label='y(t) actual', linewidth=2)
ax1_B.plot(time, yd_hist_B, '--', label='y(t) desired', linewidth=2)
ax1_B.set_xlabel("Time [s]")
ax1_B.set_ylabel("Position [m]")
ax1_B.set_title("End-Effector Position vs. Time (Part B), using Jacobian Jpinv ?? (DLS or Jpinv)")
ax1_B.legend()
ax1_B.grid(True)
plt.show()

# ----------------------
# Plot the 2D trajectory for Part B
# ----------------------
fig2_B, ax2_B = plt.subplots()
ax2_B.plot(x_hist_B, y_hist_B, label='Actual Trajectory', linewidth=2)
ax2_B.plot(xd_hist_B, yd_hist_B, '--', label='Desired Trajectory', linewidth=2)
ax2_B.set_xlabel("x [m]")
ax2_B.set_ylabel("y [m]")
ax2_B.set_title("End-Effector Trajectory (Part B)")
ax2_B.legend()
ax2_B.axis('equal')
ax2_B.grid(True)
plt.show()

Download videos

In [ ]:
# ----------------------
# Save Plots and Animations
# ----------------------
# Save Problem 1, Part A plots
fig1_A.savefig('end_effector_position_vs_time_partA.png')
fig2_A.savefig('end_effector_trajectory_partA.png')

# Save Problem 1, Part B plots
fig1_B.savefig('end_effector_position_vs_time_partB.png')
fig2_B.savefig('end_effector_trajectory_partB.png')

# Save animations:
ani_A.save('manipulator_animation_partA.mp4', writer='ffmpeg')
ani_B.save('manipulator_animation_partB.mp4', writer='ffmpeg')

# --- Download files (Google Colab) ---
from google.colab import files
files.download('end_effector_position_vs_time_partA.png')
files.download('end_effector_trajectory_partA.png')
files.download('end_effector_position_vs_time_partB.png')
files.download('end_effector_trajectory_partB.png')
files.download('manipulator_animation_partA.mp4')
files.download('manipulator_animation_partB.mp4')
